In [28]:
%load_ext autoreload
%autoreload 2

import torch
from text3d2video.artifacts.anim_artifact import AnimationArtifact
from text3d2video.artifacts.texture_artifact import TextureArtifact

torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from text3d2video.pipelines.generative_rendering_pipeline import GenerativeRenderingPipeline
from text3d2video.pipelines.pipeline_utils import load_pipeline

pipe = load_pipeline(GenerativeRenderingPipeline)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
tex_tag = "human_mvlatest_Stormtrooper:latest"
tex_tag = "human_mvlatest_Deadpool:latest"
tex_tag = 'cat_statue_mvlatest_SilverCatStatue:latest'

texture_art = TextureArtifact.from_wandb_artifact_tag(tex_tag)
texture = texture_art.read_texture()

anim_tag = 'catwalk_180_20:latest'
anim_tag = 'mma_20:latest'
anim_tag = 'mv_cat_statue:latest'
anim_tag = 'ymca_20:latest'
anim_tag = 'mv_cat_statue_25:latest'

anim_art = AnimationArtifact.from_wandb_artifact_tag(anim_tag)
anim = anim_art.read_anim_seq()

In [4]:
from text3d2video.rendering import render_texture
from text3d2video.utilities.video_comparison import display_vids
from text3d2video.utilities.video_util import pil_frames_to_clip

anim_uvs = anim.render_rgb_uv_maps()
anim_renders = render_texture(
    anim.meshes, anim.cams, texture, anim.verts_uvs, anim.faces_uvs, return_pil=True
)
display_vids([pil_frames_to_clip(anim_renders), pil_frames_to_clip(anim_uvs)])

In [5]:
from text3d2video.artifacts.anim_artifact import AnimSequence
from text3d2video.util import ordered_sample_indices

# extr_indices = [0]
src_indices = [0, 7]
src_indices = [-1]
src_indices = ordered_sample_indices(anim_renders, 5).tolist()
# src_indices = [0, 2, 4, 6]

src_anim = AnimSequence(
    cams=anim.cams[src_indices],
    meshes=anim.meshes[src_indices],
    verts_uvs=anim.verts_uvs,
    faces_uvs=anim.faces_uvs,
)

In [8]:
src_tag = 'human_mv:latest'
src_tag = 'mv_cat_statue:latest'

src_anim_art = AnimationArtifact.from_wandb_artifact_tag(src_tag)
src_anim = src_anim_art.read_anim_seq()

In [12]:
from torch import Tensor
from pytorch3d.structures.meshes import join_meshes_as_batch
from pytorch3d.renderer.camera_utils import join_cameras_as_batch

mesh = anim.meshes[-1]
cam = anim.cams[-1]

offsets = Tensor([[0, 0.0, 0], [0, 0.3, 0], [0, 0.6, 0], [0, 0.9, 0]]).cuda()

cams = []
for o in offsets:
    shifted = cam.clone()
    shifted.T = shifted.T + o
    cams.append(shifted)

cams = join_cameras_as_batch(cams)
meshes = join_meshes_as_batch([mesh] * len(offsets))

src_anim = AnimSequence(
    cams=cams,
    meshes=meshes,
    verts_uvs=anim.verts_uvs,
    faces_uvs=anim.faces_uvs,
)

In [9]:
from text3d2video.utilities.video_comparison import display_vid

src_vid = pil_frames_to_clip(src_anim.render_rgb_uv_maps(), fps=1.5)
display_vid(src_vid, title="src")

In [27]:
from pathlib import Path
from torch import Generator
from text3d2video.pipelines.generative_rendering_pipeline import (
    GenerativeRenderingConfig,
)
from text3d2video.utilities.logging import H5Logger

module_paths = [
    "up_blocks.1.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.2.transformer_blocks.0.attn1",
]

extraction_modules = [module_paths[-1]]

gr_config = GenerativeRenderingConfig(
    module_paths=module_paths,
    num_inference_steps=15,
    do_pre_attn_injection=True,
    do_post_attn_injection=True,
    num_keyframes=5,
    attend_to_self_kv=True
)

torch.cuda.empty_cache()

generator = Generator(device="cuda").manual_seed(1)
kf_generator = Generator(device="cuda").manual_seed(1)

logger = H5Logger(Path("data.h5"), enabled=False)
logger.delete_data()
logger.open_write()

logger.key_greenlists["layer"] = extraction_modules

start_noise_level = 0.25

input_src_anim = None
input_src_anim = src_anim

input_texture = None
# input_texture = texture

out = pipe(
    "Shiny Metalic Cat Statue",
    anim,
    gr_config,
    src_anim=input_src_anim,
    start_noise_level=start_noise_level,
    texture=input_texture,
    logger=logger,
    kf_generator=kf_generator,
    generator=generator,
)

logger.close()

  0%|          | 0/15 [00:00<?, ?it/s]

1.0


  7%|▋         | 1/15 [00:04<01:03,  4.56s/it]

0.95


 13%|█▎        | 2/15 [00:09<01:01,  4.73s/it]

...

 87%|████████▋ | 13/15 [01:02<00:09,  4.84s/it]

0.35


 93%|█████████▎| 14/15 [01:07<00:04,  4.85s/it]

0.30000000000000004


100%|██████████| 15/15 [01:12<00:00,  4.82s/it]


In [29]:
vids = [pil_frames_to_clip(out.images)]

title = "Video"
if input_texture is not None:
    title += f" (from_noise {start_noise_level})"

titles = [title]

if out.extr_images is not None:
    duration = vids[0].duration
    n_extr = len(out.extr_images)
    fps = n_extr / duration

    extr_vid = pil_frames_to_clip(out.extr_images, fps=fps)
    vids.append(extr_vid)
    titles.append("Extraction Images")

# prepend vid
if input_texture is not None:
    vids = [pil_frames_to_clip(anim_renders)] + vids
    titles = ["Renders"] + titles

display_vids(vids, titles=titles)

In [26]:
module_paths

['up_blocks.1.attentions.0.transformer_blocks.0.attn1',
 'up_blocks.1.attentions.1.transformer_blocks.0.attn1',
 'up_blocks.1.attentions.2.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.0.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.1.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.2.transformer_blocks.0.attn1',
 'up_blocks.3.attentions.0.transformer_blocks.0.attn1']

In [37]:
from text3d2video.sd_feature_extraction import AttnLayerId

m = module_paths[1]

for m in module_paths:
    parsed = AttnLayerId.parse(m)
    print(parsed.block_index, parsed.unet_absolute_index())

1 7
1 8
1 9
2 10
2 11
2 12
3 13
3 14
3 15


In [40]:
# vid_art = VideoArtifact.create_empty_artifact("extr_images")
# vid_art.write_frames(out.extr_images)
# vid_art.log_standalone()

extr_anim_art = AnimationArtifact.create_empty_artifact("extr_anim")
extr_anim_art.write_anim_seq(src_anim)
extr_anim_art.log_standalone()

wandb: Adding directory to artifact (/tmp/local_artifacts/animation/extr_anim)... Done. 0.1s


In [22]:
from text3d2video.backprojection import compute_texel_projections, project_views_to_video_texture
import torchvision.transforms.functional as TF

texture_res = 600
projections = compute_texel_projections(
    anim.meshes, anim.cams, anim.verts_uvs, anim.faces_uvs, texture_res=texture_res
)

images_pt = torch.stack([TF.to_tensor(x) for x in out.images]).cuda()
frames_uv = project_views_to_video_texture(images_pt, texture_res, projections).cpu()
frames_uv_pil = [TF.to_pil_image(x) for x in frames_uv]
uv_vid = pil_frames_to_clip(frames_uv_pil)

video = pil_frames_to_clip(out.images)

display_vids([video, uv_vid], titles=["Video", "UV Video"])